In [10]:
import random
import hashlib
import binascii
import sys

"""Generation of parameters and initial values"""
# STEP 1
# input:
# n - the number of iterations of a one-way function
# k - number of logins
# output:
# xA - a secret random natural number from the range <100, 1000000> generated by the user A
# xB - a secret random natural number from the range <100, 1000000> generated by the user B
# xC - a secret random natural number from the range <100, 1000000> generated by the user C
# tA - an array of n + 1 elements with consecutive values of the one-way function calculated by the user A
# tB - an array of n + 1 elements with consecutive values of the one-way function calculated by the user B
# tC - an array of n + 1 elements with consecutive values of the one-way function calculated by the user C
# sxA - identifier xn that the system will receive from user A
# sxB - identifier xn that the system will receive from user B
# sxC - identifier xn that the system will receive from user C

n = 5
k = 4
xA = random.randint(100, 1000000)
xB = random.randint(100, 1000000)
xC = random.randint(100, 1000000)
print("User A random value: ", xA)
print("User B random value: ", xB)
print("User C random value: ", xC)
xAs = str(xA)
xBs = str(xB)
xCs = str(xC)
tA = []
tB = []
tC = []

def fill(t, x, n):
    t.append(x)
    for i in range(1, n+1):
        d = hashlib.sha3_512()
        d.update(t[i-1].encode())
        hash = d.hexdigest()
        t.append(hash)
    return t

tA = fill(tA, xAs, n)
tB = fill(tB, xBs, n)
tC = fill(tC, xCs, n)
print("\nTable of user A")
for i in range(0, n+1):
    print("[", i, "]: ", tA[i])
print("\nTable of user B")
for i in range(0, n+1):
    print("[", i, "]: ", tB[i])
print("\nTable of user C")
for i in range(0, n+1):
    print("[", i, "]: ", tC[i])
print("\nUsers send xn values to the system (i.e. the last values from the array)\n")
sxA = tA[n]
sxB = tB[n]
sxC = tC[n]

User A random value:  186522
User B random value:  705118
User C random value:  580863

Table of user A
[ 0 ]:  186522
[ 1 ]:  9d29528ba94cc55e75ce57aec5fffaf45047dc02bfbf089f1737fab90d708dcc50da300eccc4d50b0f961c06825aea8e79fa40f691811f01caea2767f20d7d6e
[ 2 ]:  2cb5a17f35798257a27bb7a46c7ca673d56e964883ca48f82613d8d81afb107fa742f51f082f3cf3191013ab86566f3c4f828a1c7334ff76c84038d5472bb522
[ 3 ]:  8f9348cf52e416223c1263812bf09bb5cf89df091caacca095fd397c39b4f10e9d8477cc4b177bbe24258d3751a300c38de3be3ac740fdf9c0cedf06a0c3aaa9
[ 4 ]:  ced8f67e6aaaf06deac7f8602378351a24ac3e5dfcad578e7a8965bcd5731271829ab6ed08a08a54ad78da31f0cd31c22f71dcf48e762d1d7caf036880bf8251
[ 5 ]:  a0eaedcf018f28d95670788bccb7ee180d6c02aebe6a105877d770e2f2f714f929bc5287a6f9a7dbe8c0214c6ce39a41d1fa9ad660aaf8d5b4ce494339b03265

Table of user B
[ 0 ]:  705118
[ 1 ]:  d3112e75be7d18967d8c5a69aabcbe4bf8102e2de79c6d8a1b338961959c8f1387644f2472498ce3c44568c141fda5f057d43bc4209beda649dfd1c5e95f476d
[ 2 ]:  d1d9fc923f713419b5f

In [11]:
"""User login to the system"""
# STEP 2
# input:
# k - number of logins
# tA - an array of n + 1 elements with consecutive values of the one-way function calculated by the user A
# tB - an array of n + 1 elements with consecutive values of the one-way function calculated by the user B
# tC - an array of n + 1 elements with consecutive values of the one-way function calculated by the user C
# sxA - identifier xn received by the system from user A
# sxB - identifier xn received by the system from user B
# sxC - identifier xn received by the system from user C
# z - number of active users
# output:
# sh - value of the user one-way function, calculated by the system based on the password provided by the user
# password - the password that the system got from the user
# tk - table of the number of logins of individual users
# ts - table with current user logins

z=3
def login(k, n, tA, tB, tC, sxA, sxB, sxC, z):
    tp = [1, 1, 2, 2, 3, 3]
    t = ["A", "B", "C"]
    ts = [sxA, sxB, sxC]
    tu = [tA, tB, tC]
    tk = [0,0,0]
    for i in range(1,(k+1)):
        # the system checks if there are users who can log in
        if z > 0:
            xp = random.randint(0, 5)
            # x - variable storing the number of the logging in user
            x = tp[xp]
            print("User logs in: ", t[x-1])
            # the system asks the selected user for login
            print("User ", t[x-1], "enter the login")
            # the drawn user provides login
            print("My login is: ", tu[x-1][n-tk[x-1]])
            # the system checks the correctness of the login with the login in the system
            if ts[x-1] == tu[x-1][n-tk[x-1]]:
                print("User ", t[x-1], "enter the password")
                print("My password is: ", tu[x-1][n-1-tk[x-1]])
                password = tu[x-1][n-1-tk[x-1]]
                # the system checks the correctness of the password (f (passwords) = login)
                d = hashlib.sha3_512()
                d.update(password.encode())
                sh = d.hexdigest()
                if (ts[x-1]==sh):
                    # the password is correct, the identifier in the system is changed to the current password
                    print("Logged in\n")
                    ts[x-1]=password
                    tk[x-1]=tk[x-1]+1
                else:
                    print("User ", t[x-1], "you are a fraud\n")
                    if(z == 3):
                        # the numbers of the blocked user are replaced by the other two (x is the user number to be blocked)
                        tp[2*x-2] = x % 3 + 1
                        tp[2*x-1] = (x+1) % 3 + 1
                    if(z == 2):
                        # the entire table is filled with the numbers of the only user with access rights
                        if(tp[1] == x):
                            for g in range(0,6):
                                tp[g] = tp[2]
                        else:
                            for g in range(0,6):
                                tp[g] = tp[1]
                    # the number of active users is decreasing
                    z=z-1
            else:
                # login is not valid then end of logins for this user
                print("User ", t[x-1], "you have wrong login\n")
                if(z == 3):
                    # analogous modification of the tp table in relation to a wrong password
                    tp[2*x-2] = x % 3 + 1
                    tp[2*x-1] = (x+1) % 3 + 1
                if(z == 2):
                    if(tp[1] == x):
                        for g in range(0,6):
                            tp[g] = tp[2]
                    else:
                        for g in range(0,6):
                            tp[g] = tp[1]
                # the number of active users is decreasing
                z=z-1
        else:
            print("There are no users who can log in, system end")
            sys.exit(0)

login(k, n, tA, tB, tC, sxA, sxB, sxC, z)

User logs in:  A
User  A enter the login
My login is:  a0eaedcf018f28d95670788bccb7ee180d6c02aebe6a105877d770e2f2f714f929bc5287a6f9a7dbe8c0214c6ce39a41d1fa9ad660aaf8d5b4ce494339b03265
User  A enter the password
My password is:  ced8f67e6aaaf06deac7f8602378351a24ac3e5dfcad578e7a8965bcd5731271829ab6ed08a08a54ad78da31f0cd31c22f71dcf48e762d1d7caf036880bf8251
Logged in

User logs in:  C
User  C enter the login
My login is:  e2255bb7585d50bdddc1fae1320468979a3a38d26e0cee8b34e3406448889ecf74c0e8d840a52dfd27fbe1f93fc0a688c39f6ff567cec7b4331875afc0b65c97
User  C enter the password
My password is:  f8f10973846fa6fff6c7ddb0133900d4ffc4fb39c1ad977697ba2937037f8f259c74027584fa7752dc67d59acfea930fdea5354692b2b4365b568013f25f9ac3
Logged in

User logs in:  A
User  A enter the login
My login is:  ced8f67e6aaaf06deac7f8602378351a24ac3e5dfcad578e7a8965bcd5731271829ab6ed08a08a54ad78da31f0cd31c22f71dcf48e762d1d7caf036880bf8251
User  A enter the password
My password is:  8f9348cf52e416223c1263812bf09bb5cf89